In [1]:
import numpy as np

In [2]:
# Sioux Falls network data
# http://www.bgu.ac.il/~bargera/tntp/

#Link travel time = free flow time * ( 1 + B * (flow/capacity)^Power ).
#Link generalized cost = Link travel time + toll_factor * toll + distance_factor * distance

# Traffic Assignment Data structure

class TA_Data(object):
    
    def __init__(self, network_name, \
                 number_of_zones, number_of_nodes, number_of_links, \
                 start_node, end_node, capacity, free_flow_time, \
                 B, power, total_od_flow, travel_demand, od_pairs):
        self.network_name = network_name
        self.number_of_zones = number_of_zones
        self.number_of_nodes = number_of_nodes
        self.number_of_links = number_of_links
        self.start_node = start_node
        self.end_node = end_node
        self.capacity = capacity
        self.free_flow_time = free_flow_time
        self.B = B
        self.power = power
        self.total_od_flow = total_od_flow
        self.travel_demand = travel_demand
        self.od_pairs = od_pairs

In [3]:
def load_ta_network(network_name):
    if network_name == 'Sioux Falls':
        network_data_file = './data/SiouxFalls_net.txt'
        trip_table_file = './data/SiouxFalls_trips.txt' 
    else:
        raise Exception("Invalid network_name; use 'Sioux Falls'.")

    ################
    # Network Data
    ################
    start_node = []
    end_node = []
    capacity = []
    free_flow_time = []
    B = []
    power = []

    with open(network_data_file) as network_data_file_:
        for line in network_data_file_:
            if '<NUMBER OF ZONES>' in line:
                number_of_zones = [int(s) for s in line.split() if s.isdigit()][0]
            elif '<NUMBER OF NODES>' in line:
                number_of_nodes = [int(s) for s in line.split() if s.isdigit()][0]
            elif '<NUMBER OF LINKS>' in line:
                number_of_links = [int(s) for s in line.split() if s.isdigit()][0]
                assert(number_of_links > 0)
            elif '<FIRST THRU NODE>' in line or '<END OF METADATA>' in line or '~' in line or line == '\n':
                continue
            else:
                start_node.append(int(line.split()[0]))
                end_node.append(int(line.split()[1]))
                capacity.append(float(line.split()[2]))
                free_flow_time.append(float(line.split()[4]))
                B.append(float(line.split()[5]))
                power.append(int(line.split()[6]))

    ##############
    # Trip Table
    ##############
    travel_demand = np.zeros((number_of_zones, number_of_zones))
    od_pairs = []

    with open(trip_table_file) as trip_table_file_:
        for line in trip_table_file_:
            if '<NUMBER OF ZONES>' in line:
                number_of_zones_trip = [int(s) for s in line.split() if s.isdigit()][0]
                # Check if number_of_zone is same in both txt files
                assert(number_of_zones_trip == number_of_zones) 
            elif '<TOTAL OD FLOW>' in line:
                total_od_flow = [float(s) for s in line.split() if s.replace('.','',1).isdigit()][0]
                assert(total_od_flow > 0)
            elif '<END OF METADATA>' in line or line == '\n':
                continue
            else:
                if 'Origin' in line:
                    origin = int(line.split()[1])
                elif ';' in line:
                    pairs = line.split(';')
                    for i in range(len(pairs)):
                        if ':' in pairs[i]:
                            pair = pairs[i].split(':')
                            destination = int(pair[0])
                            od_flow = float(pair[1])
                            travel_demand[origin-1, destination-1] = od_flow
                            od_pairs.append((origin, destination))

    ta_data = TA_Data(
    network_name,
    number_of_zones,
    number_of_nodes,
    number_of_links,
    start_node,
    end_node,
    capacity,
    free_flow_time,
    B,
    power,
    total_od_flow,
    travel_demand,
    od_pairs)

    return ta_data


In [4]:
ta_data = load_ta_network('Sioux Falls')

In [99]:
import networkx as nx

In [108]:
def create_graph(start_node, end_node):
    
    assert(len(start_node) == len(end_node))

    no_node = max(max(start_node), max(end_node))
    no_arc = len(start_node)

    graph = nx.DiGraph()
    
    graph.add_nodes_from(range(no_node + 1)[1:])
    
    for i in range(no_arc):
        graph.add_edge(start_node[i], end_node[i])
        
    return graph

In [109]:
graph = create_graph(ta_data.start_node, ta_data.end_node)

In [110]:
[(ta_data.start_node[i], ta_data.end_node[i]) for i in range(len(ta_data.start_node))]

[(1, 2),
 (1, 3),
 (2, 1),
 (2, 6),
 (3, 1),
 (3, 4),
 (3, 12),
 (4, 3),
 (4, 5),
 (4, 11),
 (5, 4),
 (5, 6),
 (5, 9),
 (6, 2),
 (6, 5),
 (6, 8),
 (7, 8),
 (7, 18),
 (8, 6),
 (8, 7),
 (8, 9),
 (8, 16),
 (9, 5),
 (9, 8),
 (9, 10),
 (10, 9),
 (10, 11),
 (10, 15),
 (10, 16),
 (10, 17),
 (11, 4),
 (11, 10),
 (11, 12),
 (11, 14),
 (12, 3),
 (12, 11),
 (12, 13),
 (13, 12),
 (13, 24),
 (14, 11),
 (14, 15),
 (14, 23),
 (15, 10),
 (15, 14),
 (15, 19),
 (15, 22),
 (16, 8),
 (16, 10),
 (16, 17),
 (16, 18),
 (17, 10),
 (17, 16),
 (17, 19),
 (18, 7),
 (18, 16),
 (18, 20),
 (19, 15),
 (19, 17),
 (19, 20),
 (20, 18),
 (20, 19),
 (20, 21),
 (20, 22),
 (21, 20),
 (21, 22),
 (21, 24),
 (22, 15),
 (22, 20),
 (22, 21),
 (22, 23),
 (23, 14),
 (23, 22),
 (23, 24),
 (24, 13),
 (24, 21),
 (24, 23)]

In [111]:
graph.edges()

[(1, 2),
 (1, 3),
 (2, 1),
 (2, 6),
 (3, 1),
 (3, 4),
 (3, 12),
 (4, 11),
 (4, 3),
 (4, 5),
 (5, 9),
 (5, 4),
 (5, 6),
 (6, 8),
 (6, 2),
 (6, 5),
 (7, 8),
 (7, 18),
 (8, 16),
 (8, 9),
 (8, 6),
 (8, 7),
 (9, 8),
 (9, 10),
 (9, 5),
 (10, 16),
 (10, 9),
 (10, 11),
 (10, 17),
 (10, 15),
 (11, 12),
 (11, 10),
 (11, 4),
 (11, 14),
 (12, 11),
 (12, 3),
 (12, 13),
 (13, 24),
 (13, 12),
 (14, 23),
 (14, 11),
 (14, 15),
 (15, 10),
 (15, 19),
 (15, 22),
 (15, 14),
 (16, 8),
 (16, 17),
 (16, 10),
 (16, 18),
 (17, 16),
 (17, 10),
 (17, 19),
 (18, 16),
 (18, 20),
 (18, 7),
 (19, 17),
 (19, 20),
 (19, 15),
 (20, 18),
 (20, 19),
 (20, 21),
 (20, 22),
 (21, 24),
 (21, 20),
 (21, 22),
 (22, 23),
 (22, 20),
 (22, 21),
 (22, 15),
 (23, 24),
 (23, 22),
 (23, 14),
 (24, 23),
 (24, 13),
 (24, 21)]